In [1]:
#verification that each region yileds the equilibrium suggested by theory
%load_ext autoreload
%autoreload 2
from utils.imports import *
from utils.single_stage import *
from utils.single_stage_old import get_payoffs_a,get_payoffs_b,get_computed_equilibrium,constraints_a,constraints_b

In [2]:
dist  = 'uniform'
npts = 20
ca,cb,la,lb = ml_get_example_in_region(3,dist)
maxpx = ca+5

{'ca': 1, 'cb': 0, 'la': 2, 'lb': 2}
{'paa_sst': 1.98, 'pba_sst': 0.99, 'pab_sst': 1.32, 'pbb_sst': 1.65}


In [3]:
paa_arr,pba_arr,objaa,objba,constraint_a_arr = ml_get_payoff_matrices_state_a(ca,cb,maxpx,npts,dist,la)
pab_arr,pbb_arr,objab,objbb,constraint_b_arr = ml_get_payoff_matrices_state_b(ca,cb,maxpx,npts,dist,lb)

In [4]:
compute_single_stage_equilibrium(objaa,objba,objbb,objab,paa_arr,pba_arr,pbb_arr,pab_arr)

{'paa': 2.053,
 'pba': 0.947,
 'pbb': 1.579,
 'pab': 1.263,
 'objaa': 0.471,
 'objba': 0.524,
 'objbb': 1.33,
 'objab': 0.042}

In [5]:
paa_arr,pba_arr,objaa,objba,cmata = get_payoffs_a(ca,cb,maxpx,npts,dist,la)
pbb_arr,pab_arr,objbb,objab,cmatb = get_payoffs_b(ca,cb,maxpx,npts,dist,lb)

paa_c,pba_c,objaa_c,objba_c = get_computed_equilibrium([np.array([objaa,objba])],paa_arr,pba_arr)
pbb_c,pab_c,objbb_c,objab_c = get_computed_equilibrium([np.array([objbb,objab])],pbb_arr,pab_arr)

# print('A\'s strong sub-market (theoretical:paa,pba,objaa,objba,constraintA):',
#       paa_t,pba_t,objaa_t,objba_t,constraints_a(paa_t,pba_t,ca,cb,la))
print('A\'s strong sub-market (computed:paa,pba,objaa,objba,constraintA):',
      paa_c,pba_c,objaa_c,objba_c,constraints_a(paa_c,pba_c,ca,cb,la))
# print('B\'s strong sub-market (theoretical:pbb,pab,objbb,objab,constraintB):',
#       pbb_t,pab_t,objbb_t,objab_t,constraints_b(pbb_t,pab_t,cb,ca,lb))
print('B\'s strong sub-market (computed:pbb,pab,objbb,objab,constraintB):',
      pbb_c,pab_c,objbb_c,objab_c,constraints_b(pbb_c,pab_c,cb,ca,lb))

A's strong sub-market (computed:paa,pba,objaa,objba,constraintA): 2.053 0.947 0.471 0.524 True
B's strong sub-market (computed:pbb,pab,objbb,objab,constraintB): 1.579 1.263 1.33 0.042 True
